In [ ]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from whoosh.qparser import QueryParser

from thesiscode import references
import aliases

schema = Schema(
    **{key: TEXT(stored = True) for key in allkeys}
    )

ix = create_in("indexdir", schema)
writer = ix.writer()

biblio = references.parse_bibtex('references', aliases.bookdir)
allkeys = set.union(*list(set(entry.keys()) for entry in biblio.entries))

for entry in biblio.entries:
    writer.add_document(**entry)
writer.commit()

def print_hit(hit):
    print('\n')
    print('-' * 79)
    print(hit['ID'])
    print(hit['title'])
    print(hit['author'])
    print('-' * 79)
    if 'abstract' in hit:
        print('Abstract:')
        for line in hit['abstract'].split('\n'):
            print(line)
    if 'keywords' in hit:
        print('Keywords:' + hit['keywords'])
    print('-' * 79)
    print('\n')

def search(strn, **kwargs):
    with ix.searcher() as searcher:
        parser = QueryParser("abstract", ix.schema)
        hits = searcher.search(parser.parse(strn), **kwargs)
        for hit in hits:
            print_hit(hit)